# ConversationEntityMemory

엔티티 메모리는 대화에서 특정 엔티티에 대한 주어진 사실을 기억합니다.

엔티티 메모리는 엔티티에 대한 정보를 추출하고(LLM 사용) 시간이 지남에 따라 해당 엔티티에 대한 지식을 축적합니다(역시 LLM 사용).


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# from langchain_openai import ChatOpenAI
# from langchain.chains import ConversationChain
# from langchain.memory import ConversationEntityMemory
# from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
# 메모리 클래스를 제공하지 않고 이제는 직접 메모리 저장을 구현하는 방식으로 바뀜

# 엔티티 추출
# 엔티티를 key-value 저장소에 보관
# 다음 대화 시 prompt에 해당 엔티티 정보를 주입

from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
import json


c:\Users\jeffr\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-SGRq4Tjy-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Entity 메모리를 효과적으로 사용하기 위하여, 제공되는 프롬프트를 사용합니다.


In [7]:
# # Entity Memory를 사용하는 프롬프트 내용을 출력합니다.
# print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)

entity_store = {}

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

entity_extract_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "아래 문장에서 등장하는 엔티티(사람, 직업, 조직, 관계 등)를 JSON으로 추출하세요.\n"
     "가능한 key: person, job, org, relation 등"),
    ("human", "{text}")
])

entity_extract_chain = entity_extract_prompt | llm


def extract_entities(text: str):
    #LLM을 사용해 엔티티 JSON을 추출
    result = entity_extract_chain.invoke({"text": text})
    try:
        return json.loads(result.content)
    except:
        return {}


def update_entity_store(text: str):
    #추출한 엔티티를 저장소에 업데이트
    entities = extract_entities(text)
    for k, v in entities.items():
        entity_store[k] = v

conversation_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "현재까지 발견된 엔티티 정보는 다음과 같습니다.\n"
     "{entity_memory}\n\n"
     "이 정보를 참고하여 사용자의 메시지에 대답하세요."),
    MessagesPlaceholder(variable_name="messages")
])

main_chain = conversation_prompt | llm


In [4]:
# # LLM 을 생성합니다.
# llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

# # ConversationChain 을 생성합니다.
# conversation = ConversationChain(
#     llm=llm,
#     prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
#     memory=ConversationEntityMemory(llm=llm),
# )


def entity_memory_wrapper(inputs):
    msgs = inputs.get("messages", [])
    
    # RunnableWithMessageHistory 내부 호출 방지
    if not isinstance(msgs, list) or len(msgs) == 0:
        return inputs  # 메시지가 없을 때는 아무 것도 하지 않음

    # 사용자 메시지에서 엔티티 추출
    user_text = msgs[-1].content
    update_entity_store(user_text)

    # prompt 변수 주입
    inputs["entity_memory"] = json.dumps(entity_store, ensure_ascii=False)
    return inputs


prompt = ChatPromptTemplate.from_messages([
    ("system", "엔티티 메모리를 참고하여 대화를 이어가세요."),
    MessagesPlaceholder(variable_name="messages")
])

# 3. 메모리 (ConversationEntityMemory 대체)
history = InMemoryChatMessageHistory()


conversation = RunnableWithMessageHistory(
    main_chain,
    lambda session_id: history,
    input_messages_key="messages",
    history_messages_key="messages"
)




대화를 시작합니다.

입력한 대화를 바탕으로 `ConversationEntityMemory` 는 주요 Entity 정보를 별도로 저장합니다.


In [9]:
# conversation.predict(
#     input="테디와 셜리는 한 회사에서 일하는 동료입니다."
#     "테디는 개발자이고 셜리는 디자이너입니다. "
#     "그들은 최근 회사에서 일하는 것을 그만두고 자신들의 회사를 차릴 계획을 세우고 있습니다."
# )

user_input = (
    "테디와 셜리는 한 회사에서 일하는 동료입니다. "
    "테디는 개발자이고 셜리는 디자이너입니다. "
    "그들은 최근 회사를 그만두고 창업을 준비하고 있습니다."
)

user_messages = [HumanMessage(content=user_input)]

wrapper_input = {"messages": user_messages}
wrapper_output = entity_memory_wrapper(wrapper_input)

response = conversation.invoke(
    {"messages": user_messages,
     "entity_memory": wrapper_output["entity_memory"]},
    config={"configurable": {"session_id": "entity_session"}}
)

Entity는 `memory.entity_store.store` 에서 확인할 수 있습니다.


In [10]:
# # entity memory 를 출력합니다.
# conversation.memory.entity_store.store

print("=== 모델 응답 ===")
print(response.content)

print("\n=== 엔티티 저장소 ===")
print(entity_store)

=== 모델 응답 ===
테디는 개발자이고, 셜리는 디자이너입니다. 두 사람은 한 회사에서 함께 일하는 동료입니다. 도움이 필요하시면 언제든지 말씀해 주세요!

=== 엔티티 저장소 ===
{'person': ['테디', '셜리'], 'job': {'테디': '개발자', '셜리': '디자이너'}, 'relation': {'테디-셜리': '동료'}, 'org': ['한 회사']}
